# Figure 10: Glacier signature uncertainty

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

import os
from glob import glob
from oggm import utils

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

cl = px.colors.qualitative.D3
os.chdir('/home/rooda/Dropbox/Patagonia/')

## Data

In [ ]:
# basin data
basins = gpd.read_file("GIS South/Basins_Patagonia_ice.shp")
basins = basins.set_index("ID")

basin_names = ["Yelcho", "Baker", "Santa Cruz", "Palena", "Grey", "Puelo", "Cisnes", "Aysen", "Pascua"] 
basins.loc[basins.basin_area > 5000, "Name"] = basin_names
basins = basins.dropna(subset = ['Name'])

In [ ]:
# feature importance data
data = pd.read_csv("MS2 Results/feature_importance_rmse.csv", index_col = "ID")
data_main = data[data.index.isin(basins.index)]

In [ ]:
metrics = data.Metric.unique()
metrics = metrics.reshape(2,5)

titles = ['Reference magnitude', 'Peak water year', 'Peak water magnitude', 'Peak water duration', 'Inter-annual variability', 
          'Seasonal contribution', 'Seasonal variability', 'Seasonal shift', 'Long-term trend', 'Long-term change']

fig = make_subplots(rows=2, cols=5, horizontal_spacing = 0.01, vertical_spacing = 0.07, shared_yaxes = True, 
                    subplot_titles = titles, x_title='Feature importance (% of RMSE loss after permutations)')

# a to e) Boxplot -----------------------------------------------------------------------------------------------------------
for y in range (0,2):
    for x in range(0,5):   
        
        # melt on glacier
        data_xy = data[(data.Metric == metrics[y,x]) & (data.Variable == "melt_on_glacier")]
        data_main_xy = data_main[(data_main.Metric == metrics[y,x]) & (data_main.Variable == "melt_on_glacier")]

        fig.add_trace(go.Box(x = data_xy.Outline, name = "Geometry",  marker_color=cl[3], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=True), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.Volume,  name = "Volume",    marker_color=cl[1], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=True), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.Climate, name = "Climate",   marker_color=cl[2], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=True), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.GCM,     name = "GCM",       marker_color=cl[0], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=True), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.SSP,     name = "SSP",       marker_color=cl[4], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=True), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.BCM,     name = "BCM",       marker_color=cl[5], marker = dict(size= 3), opacity=0.4, offsetgroup= "A", showlegend = False, boxmean=True), row=y+1, col=x+1)
  
        # total runoff
        data_xy = data[(data.Metric == metrics[y,x]) & (data.Variable == "total_runoff")]
        data_main_xy = data_main[(data_main.Metric == metrics[y,x]) & (data_main.Variable == "total_runoff")]

        fig.add_trace(go.Box(x = data_xy.Outline, name = "Geometry",  marker_color=cl[3], marker = dict(size= 3), opacity=0.8, offsetgroup= "b", showlegend = False, boxmean=True), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.Volume,  name = "Volume",    marker_color=cl[1], marker = dict(size= 3), opacity=0.8, offsetgroup= "b", showlegend = False, boxmean=True), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.Climate, name = "Climate",   marker_color=cl[2], marker = dict(size= 3), opacity=0.8, offsetgroup= "b", showlegend = False, boxmean=True), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.GCM,     name = "GCM",       marker_color=cl[0], marker = dict(size= 3), opacity=0.8, offsetgroup= "b", showlegend = False, boxmean=True), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.SSP,     name = "SSP",       marker_color=cl[4], marker = dict(size= 3), opacity=0.8, offsetgroup= "b", showlegend = False, boxmean=True), row=y+1, col=x+1)
        fig.add_trace(go.Box(x = data_xy.BCM,     name = "BCM",       marker_color=cl[5], marker = dict(size= 3), opacity=0.8, offsetgroup= "b", showlegend = False, boxmean=True), row=y+1, col=x+1)
    
fig.add_annotation(text="Total runoff", font=dict(size=12),  arrowhead=0, arrowwidth=1, arrowcolor="rgb(30,30,30)", ax=40, x=0.05, y=4.2, showarrow=True, row=1, col=1)
fig.add_annotation(text="Melt on glacier", font=dict(size=12),  arrowhead=0, arrowwidth=1, arrowcolor="rgb(30,30,30)", ax=70, x=0.05, y=3.8, showarrow=True, row=1, col=1)

fig.update_xaxes(range = [-0.12, 0.85], row = 1, col = 2)
fig.update_xaxes(range = [-0.15, 0.8], row = 2, col = 2)
fig.update_xaxes(range = [-0.1, 0.6], row = 2, col = 3)
fig.update_xaxes(range = [-0.20, 0.8], row = 2, col = 4)
fig.update_xaxes(range = [-0.05, 0.7], row = 2, col = 5)

# general 
fig.update_layout(boxmode='group', boxgap=0.2)
fig.update_layout(plot_bgcolor="rgba(213,213,213,0.6)")
fig.update_layout(width = 900, height =900, template = "seaborn", autosize = False, margin = dict(l=10, r=10, b=50, t=30, pad=0, autoexpand=True))
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=False, showline = True, linecolor = 'black', linewidth = 1, tickformat = ',.0%', dtick = 0.3, ticks="outside", mirror=True)
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=True,  showline = True, linecolor = 'black', linewidth = 1, mirror=True)

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_10_signature.png", scale=4)
fig.show()